In [91]:
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout, Dense, Flatten, BatchNormalization, LSTM, Input, concatenate
from keras.models import Sequential, Model
from keras.optimizers import Nadam

# Training

In [112]:
data = np.ones((10,4))
data[1] = np.zeros((1, 4))
data.shape

(10, 4)

In [110]:
from nltk.util import ngrams
look_back = 5
output = np.asarray(list(ngrams(data, look_back)))
tok = np.zeros((6,3))
output.shape, tok.shape

((6, 5, 4), (6, 3))

In [111]:
y = np.zeros((output.shape[0], 15))
y[:,0] = 1
y.shape

(6, 15)

In [104]:
def build():
    """""
    model = Sequential()
    model.add(LSTM(10, 
                   input_shape=input_shape[1],
                   return_sequences = False))
    model.add(LSTM(10, return_sequences = True))
    model.add(LSTM(10))
    model.add(Dense(15, activation='softmax')) # 15 different types of note combinations (0000 is removed)"""""
    arrows = Input(shape = (5,4,))
    tokens = Input(shape = (3,))
    x = LSTM(10, return_sequences = True)(arrows)
    x = LSTM(10, return_sequences = True)(x)
    x = LSTM(10)(x)
    x = Model(inputs = arrows, outputs = x)
    y = Dense(1, activation = "relu")(tokens)
    y = Model(inputs = tokens, outputs = y)
    combined = concatenate([x.output, y.output])
    z = Dense(15, activation = "softmax")(combined)
    model = Model(inputs = [x.input, y.input], outputs = z)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    print(model.summary())
    return model

In [106]:
model = build()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 5, 4)         0                                            
__________________________________________________________________________________________________
lstm_27 (LSTM)                  (None, 5, 10)        600         input_3[0][0]                    
__________________________________________________________________________________________________
lstm_28 (LSTM)                  (None, 5, 10)        840         lstm_27[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
lstm_29 (L

In [109]:
batch_size = 1
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
model.fit([output, tok], 
          y,
          epochs = 20,
          batch_size= batch_size,
          verbose = 1)

Epoch 1/20
6/6 [==============================] - 4s 652ms/step - loss: 2.6549 - acc: 0.8333
Epoch 2/20
6/6 [==============================] - 0s 42ms/step - loss: 2.5398 - acc: 1.0000
Epoch 3/20
6/6 [==============================] - 0s 40ms/step - loss: 2.3466 - acc: 1.0000
Epoch 4/20
6/6 [==============================] - 0s 38ms/step - loss: 1.9894 - acc: 1.0000
Epoch 5/20
6/6 [==============================] - 0s 37ms/step - loss: 1.3709 - acc: 1.0000
Epoch 6/20
6/6 [==============================] - 0s 37ms/step - loss: 0.6783 - acc: 1.0000
Epoch 7/20
6/6 [==============================] - 0s 38ms/step - loss: 0.2809 - acc: 1.0000
Epoch 8/20
6/6 [==============================] - 0s 41ms/step - loss: 0.1336 - acc: 1.0000
Epoch 9/20
6/6 [==============================] - 0s 38ms/step - loss: 0.0791 - acc: 1.0000
Epoch 10/20
6/6 [==============================] - 0s 38ms/step - loss: 0.0571 - acc: 1.0000
Epoch 11/20
6/6 [==============================] - 0s 37ms/step - loss: 0.0466

# Predicting